In [ ]:
!pip install -q -U transformers datasets

In [ ]:
# Load the dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Extract the string "train" from the output and print
output_str = str(list(dataset.keys())[0]) # train
# train
k=dataset[output_str].column_names  # LIST of columns

inputs=[f'{dataset[output_str][k[0]]}: {X} Label :'for X in dataset[output_str][k[0]] ]

# TASK  all list columns  make procesor pre-processing

In [ ]:
inputs[0]

In [ ]:

# Load the dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Extract the string "train" from the output and print
output_str = str(list(dataset.keys())[0])
# train
k=dataset[output_str].column_names





def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_length, truncation=True, padding='max_length', add_special_tokens=False)

    model_inputs["labels"] = [
        [-100 if input_id == tokenizer.pad_token_id else label_id for input_id, label_id in zip(input_seq, label_seq)]
        for input_seq, label_seq in zip(model_inputs["input_ids"], labels["input_ids"])
    ]

    return model_inputs

#list[column, column]

In [ ]:
# data preprocessing
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from datasets import load_dataset


tokenizer = AutoTokenizer.from_pretrained('gpt2')
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def detect_text_and_label_columns(dataset,split:str):
    """
    This function attempts to automatically detect the text and label columns.
    The heuristic used here is to find the two longest string columns.
    """
    # Get the list of string columns in the dataset
    string_columns = [k for k, v in dataset['train'].features.items() if isinstance(v, datasets.Value) and v.dtype == 'string']

# Check if each column exists in the dataset before sorting
    sorted_columns = sorted([col for col in string_columns if col in dataset['train'].column_names],
                        key=lambda col: max(len(x) for x in dataset['train'][col]),
                        reverse=True)

    # if len(sorted_columns) < 2:
    #     raise ValueError("Unable to automatically detect text and label columns. Please specify them manually.")
    return  sorted_columns


def preprocess_dataset(dataset_name):
    # Load a dataset
    dataset = load_dataset(dataset_name)

    # Function to preprocess an example based on detected feature types
    def preprocess_function(example):
        new_example = {}
        # Loop over all features in the example
        for feature_name, feature_value in example.items():
            if isinstance(feature_value, str):  # For text features
                # Apply text preprocessing like tokenization, lowercasing, etc.
                new_example[feature_name] = feature_value.lower()  # Example operation
            elif isinstance(feature_value, list):  # For categorical features
                if isinstance(feature_value[0], int):  # Label encoded list
                    # Convert numeric labels to strings if names are provided in the feature
                    label_feature = dataset["train"].features[feature_name]
                    if hasattr(label_feature, 'names'):
                        new_example[feature_name] = [label_feature.names[label] for label in feature_value]
                    else:
                        new_example[feature_name] = feature_value  # Unchanged
                else:
                    # Handle other list data types if necessary
                    pass
            else:  # For numerical or other types of features
                # Apply numerical preprocessing like normalization, scaling, etc.
                new_example[feature_name] = feature_value  # Example operation

        return new_example

    # Apply the preprocessing function to all splits in the dataset
    processed_dataset = dataset.map(preprocess_function, batched=True, num_proc=1)

    return processed_dataset



def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_length, truncation=True, padding='max_length', add_special_tokens=False)

    model_inputs["labels"] = [
        [-100 if input_id == tokenizer.pad_token_id else label_id for input_id, label_id in zip(input_seq, label_seq)]
        for input_seq, label_seq in zip(model_inputs["input_ids"], labels["input_ids"])
    ]

    return model_inputs

# Load and preprocess the dataset
dataset_name = "LDJnr/Capybara"
processed_dataset = preprocess_dataset(dataset_name)

dataset=load_dataset("LDJnr/Capybara")
sorted_columns=detect_text_and_label_columns(dataset,'train')
len(sorted_columns)
if len(sorted_columns)==1:
    text_column =sorted_columns[0]
else:
# Define your columns for text and labels based on your dataset
  text_column =sorted_columns[0] # Replace with actual text column name
  label_column = sorted_columns[1]  # Replace with actual label column name

print(text_column )


# Additional required parameters
max_length = 512  # Example max length, adjust as needed

# Now apply your tokenizer processing function to the processed_dataset
processed_datasets = processed_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=processed_dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

# Use the processed_datasets for training
train_dataset = processed_datasets["train"][0]
print(train_dataset['input_ids'])